In [21]:
# Initialization and constants
import numpy as np
import scipy as sp
import ase
import matplotlib.pyplot as mpl

a1 =  0.297104
a2 =  1.236745
a3 =  5.749982
a4 = 38.216677

a = np.array([a1,a2,a3,a4])

C_init = np.ones(4) # 'Most initial values of C converge'
C = C_init/np.linalg.norm(C_init) # Normalization to unity

In [33]:
# Step 0: Chi 

def chi(r,i):
    '''
    Returns chi_i(r). Note that a is indexed from 1 to 4, not 0 to 3.
    '''
    return np.exp(-a[i]*r)

# Step 1: Constructing S, h and Q

S = np.array([[np.sqrt( np.pi / (p + q) ) for q in a] for p in a]) # Eq 3.26

def Q_element(p,q,r,s):
    '''
    Returns an element Q_pqrs of the Q matrix.
    '''
    q = 2*np.pi**(5/2) / ((p+q) * (r+s) * np.sqrt(p+q+r+s))
    return q

Q = np.array([[[[Q_element(p,q,r,s) for s in a] for r in a] for q in a] for p in a]) # Eq 4.17

h = see the equation written down in the notebook

In [35]:
# Testing

print(S.shape)

(4, 4)
